# 模型类

到目前为止，我们已经先后创建了张量、层、损失函数、优化器、和数据集等五个核心基础类。

现在，我们将创建最后一个核心基础类：模型。我们将用模型把层、损失函数、和优化器，以及模型训练和模型测试的逻辑，全部封装到一起。

In [72]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [73]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def size(self):
        return self.data.shape[-1]

    def __str__(self):
        return str(self.data)

### 基础数据集

In [74]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

### 基础层

In [75]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### 基础损失函数

In [76]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [77]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def clear(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

基础模型是一个抽象类，封装了两个接口：

* train()：模型训练
* test()：模型测试

In [78]:
class Model(ABC):

    def __init__(self, layer, loss, optimizer):
        self.layer = layer
        self.loss = loss
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### 数据集

In [79]:
class Dataset(Dataset):

    def load(self):
        self.train_features = [[22.5, 72.0],
                               [31.4, 45.0],
                               [19.8, 85.0],
                               [27.6, 63]]

        self.train_labels = [[95],
                             [210],
                             [70],
                             [155]]

        self.test_features = [[28.1, 58.0]]

        self.test_labels = [[165]]

## 模型

### 线性层

In [80]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data / len(x.data)
            self.bias.grad += np.sum(p.grad, axis=0) / len(x.data)

        p.gradient_fn = gradient_fn
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### 损失函数（平均平方差）

In [81]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

In [82]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经元网络模型

神经元网络模型把我们之前的训练迭代，和验证逻辑封装在一起。这样方便我们只需要简单调用就可以进行模型训练和测试。

这里我们引入**轮次**（Epoch）的概念。

In [83]:
class NNModel(Model):

    def train(self, dataset, epochs):
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                error = self.loss(predictions, labels)

                self.optimizer.clear()
                error.backward()
                self.optimizer.step()

    def test(self, dataset):
        dataset.eval()

        features, labels = dataset.items()
        predictions = self.layer(features)
        error = self.loss(predictions, labels)
        return predictions, error

## 设置

### 学习率

In [84]:
LEARNING_RATE = 0.00001

### 批大小

In [85]:
BATCH_SIZE = 2

### 轮次

In [86]:
EPOCHS = 1000

## 训练

### 迭代

通过创建一个模型，把数据集、层、损失函数、和优化器结合起来。模型训练的代码简单明了。

In [87]:
dataset = Dataset(BATCH_SIZE)
layer = Linear(dataset.shape()[0],
               dataset.shape()[1])
loss = MSELoss()
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

model = NNModel(layer, loss, optimizer)
model.train(dataset, EPOCHS)
print(layer)

weight: [[ 8.04709172 -1.08194575]]
bias: [0.15285452]


## 验证

### 测试

模型测试的代码也同样简洁。

In [88]:
predictions, error = model.test(dataset)
print(f'prediction: {predictions}')
print(f'error: {error}')

prediction: [[163.52327795]]
error: 2.1807080003282495
